In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
client_df = pd.read_csv('../data/01_raw/client_data.csv')
client_df.info()

FileNotFoundError: [Errno 2] No such file or directory: '../data/01_raw/client_data.csv'

In [ ]:
price_df = pd.read_csv('../data/01_raw/price_data.csv')
price_df.info()
price_df.head()
price_df['price_date'].value_counts()

In [ ]:
client_df['id'].value_counts().sort_index(ascending=False)

In [ ]:
price_df['id'].value_counts().sort_index(ascending=False)

In [ ]:
price_df[price_df['id'] == 'ffff7fa066f1fb305ae285bb03bf325a']

In [ ]:
# # price_df 최신데이터만 중복 제거
# price_df = price_df.sort_values('price_date', ascending=False).drop_duplicates(subset='id', keep='first')
# price_df.info()

In [ ]:
# 데이터셋 join 진행 id 기준
total_df = pd.merge(client_df, price_df, on = 'id')
total_df.info()
total_df['churn'].value_counts().sort_index()

# 결측치 없음
total_df.isna().sum()
# 중복치 없음
total_df.duplicated().sum()

# 데이터 컬럼 명세서 (Data Column Descriptions)

이 문서는 모델링에 사용되는 데이터(`data_for_model.csv`)의 컬럼들을 설명합니다.

## 1. 고객 정보 (Client Data)
| 컬럼명 | 설명 |
|---|---|
| `id` | 고객 식별자 |
| `channel_sales` | 판매 채널 코드 |
| `cons_12m` | 지난 12개월 전기 소비량 |
| `cons_gas_12m` | 지난 12개월 가스 소비량 |
| `cons_last_month` | 지난달 전기 소비량 |
| `date_activ` | 계약 활성화 날짜 |
| `date_end` | 계약 등록 마감 날짜 |
| `date_modif_prod` | 마지막 제품 수정 날짜 |
| `date_renewal` | 다음 계약 갱신 날짜 |
| `forecast_cons_12m` | 향후 12개월 예측 전기 소비량 |
| `forecast_cons_year` | 향후 1년 예측 소비량 |
| `forecast_discount_energy` | 에너지 할인 예측값 |
| `forecast_meter_rent_12m` | 향후 12개월 예측 계량기 임대료 |
| `forecast_price_energy_off_peak` | 비피크 시간대 에너지 예측 가격 |
| `forecast_price_energy_peak` | 피크 시간대 에너지 예측 가격 |
| `forecast_price_pow_off_peak` | 비피크 시간대 전력 예측 가격 |
| `has_gas` | 가스 사용 여부 (T/F) |
| `imp_cons` | 현재 지불된 소비량 |
| `margin_gross_pow_ele` | 전력 총 마진 |
| `margin_net_pow_ele` | 전력 순 마진 |
| `nb_prod_act` | 활성 제품 수 |
| `net_margin` | 총 순 마진 |
| `num_years_antig` | 고객 유지 연수 (Antiquity) |
| `origin_up` | 계약 체결 경로 (캠페인 등) |
| `pow_max` | 계약 전력량 (Power) |
| `churn` | **이탈 여부 (Target Variable)** (1: 이탈, 0: 유지) |

## 2. 가격 피처 (Price Features) - 파생 변수
가격 데이터(`price_data.csv`)를 고객 ID별로 집계하여 생성한 변수들입니다.
통계량 접미사: `_mean` (평균), `_max` (최대), `_min` (최소), `_std` (표준편차)

| 컬럼명 패턴 | 설명 |
|---|---|
| `price_off_peak_var_*` | 비피크 시간대 전력량 요금(변동) 통계 |
| `price_peak_var_*` | 피크 시간대 전력량 요금(변동) 통계 |
| `price_off_peak_fix_*` | 비피크 시간대 전력 요금(고정) 통계 |
| `price_peak_fix_*` | 피크 시간대 전력 요금(고정) 통계 |

## 3. 신규 생성 피처 (New Features)
| 컬럼명 | 설명 |
|---|---|
| `off_peak_diff` | 12월 가격 - 1월 가격 차이 (비피크 변동 요금 기준) |
| `max_volatility` | 연중 최대 가격 변동폭 (`price_off_peak_var`의 Max - Min) |
| `price_shock_encoded` | 가격 충격 그룹 인코딩 <br> `0`: Price Down (가격 하락) <br> `1`: No Change (변동 없음) <br> `2`: Price Up (가격 상승) |


In [ ]:
total_df['cons_12m'].value_counts().sort_index()

In [ ]:
# 한달 이용 전력량 (total_df['cons_12m'] / 12)
# 피크, 비피크 비율 4:6
# 기본 전력량  total_df['price_off_peak_fix'] 한달비용
# 기본 전력량  total_df['price_peak_fix'] 한달비용
month_peak_fix = total_df['price_peak_fix']
month_off_fix = total_df['price_off_peak_fix']
month_peak_var = total_df['price_peak_var']
month_off_var = total_df['price_off_peak_var']
month_use = total_df['cons_12m'] / 12

total_df['total_price'] = month_peak_fix + month_off_fix + month_peak_var * month_use * 0.4 + month_off_var * month_use * 0.6
total_df.describe()

total_df['id'].value_counts()


In [ ]:
total_df['id'][total_df['churn'] == 1].value_counts()

In [ ]:
# 이탈한 사람의 전기요금 총가격 
target_id = '2c2ff2f66da9433f98673c23526e8b3d'
df_id = total_df[total_df['id'] == target_id].sort_values('price_date')
df_id
plt.figure(figsize=(10, 4))
plt.plot(
    df_id['price_date'],
    df_id['total_price'],
    marker='o',
    linewidth=2
)
plt.xticks(rotation=45)
# plt.plot(total_df['price_date'], total_df[total_df['id'] == '563dde550fd624d7352f3de77c0cdfcd'])
plt.show()

In [ ]:
monthly_churn = (
    total_df
    .groupby(['price_date', 'churn'])['price_peak_fix']
    .mean()
    .reset_index()
)

for c in [0, 1]:
    temp = monthly_churn[monthly_churn['churn'] == c]
    plt.plot(
        temp['price_date'],
        temp['price_peak_fix'],
        label=f'churn={c}'
    )

plt.legend()
plt.xlabel('Month')
plt.ylabel('Avg Peak Fixed Charge')
plt.title('Peak Fixed Charge: Churn vs Non-Churn')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
monthly_churn = (
    total_df
    .groupby(['price_date', 'churn'])['price_off_peak_fix']
    .mean()
    .reset_index()
)

for c in [0, 1]:
    temp = monthly_churn[monthly_churn['churn'] == c]
    plt.plot(
        temp['price_date'],
        temp['price_off_peak_fix'],
        label=f'churn={c}'
    )

plt.legend()
plt.xlabel('Month')
plt.ylabel('Avg Off Peak Fixed Charge')
plt.title('Peak Off Fixed Charge: Churn vs Non-Churn')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
total_df.head()

In [ ]:
# 상관계수 매트릭스 도출
total_corr = total_df.corr(numeric_only=True)
total_corr

# 히트맵 시각화 
plt.figure(figsize=(20, 20))
sns.heatmap(total_corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

numeric_df = total_df.select_dtypes(include='number')
type(numeric_df)

X = numeric_df.drop('churn', axis=1)

X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index = X.index)
X_scaled_df.head()
total2_df = X_scaled_df
total2_df['churn'] = total_df['churn']
total2_df.head()

In [ ]:
# 상관계수 매트릭스 도출
total2_corr = total2_df.corr(numeric_only=True)
total2_corr

# 히트맵 시각화 
plt.figure(figsize=(20, 20))
sns.heatmap(total2_corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.show()